In [ ]:
import pandas as pd
from fastai.collab import *
from fastai.tabular.all import *

In [65]:
# Load the dataset into a pandas dataframe
data = pd.read_csv("../dataset/interactions_train.csv")

In [66]:
data = data[["user_id", "recipe_id", "rating"]]
print(data.head())

   user_id  recipe_id  rating
0     2046       4684     5.0
1     2046        517     5.0
2     1773       7435     5.0
3     1773        278     4.0
4     2046       3431     5.0


In [67]:
dls = CollabDataLoaders.from_df(data, bs=64)
dls.show_batch()

,user_id,recipe_id,rating
0,841895,268243,5.0
1,29782,10198,5.0
2,27060,155186,5.0
3,881750,79469,4.0
4,456641,148018,5.0
5,222433,64138,3.0
6,425154,21882,5.0
7,481092,405481,1.0
8,329138,136589,5.0
9,63128,39902,4.0


In [69]:
learn = collab_learner(dls,use_nn=True, n_factors=16, y_range=(0, 5.5))

In [ ]:
learn.fit_one_cycle(5, 5e-3, wd=0.1)

epoch,train_loss,valid_loss,time
0,0.892431,0.848385,10:18
1,0.835186,0.845575,09:52
2,0.862547,0.838719,09:56


## TensorFlow

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_recommenders as tfrs

In [9]:
data.head()

,user_id,recipe_id,rating
0,2046,4684,5.0
1,2046,517,5.0
2,1773,7435,5.0
3,1773,278,4.0
4,2046,3431,5.0


In [10]:
data = data.sample(frac=0.001).reset_index()


In [11]:
data.user_id.head().map(str)

0    497938
1    309630
2     58439
3     66847
4    415204
Name: user_id, dtype: object

In [12]:
data.shape

(699, 4)

In [13]:
user_ids_vocabulary = tf.keras.layers.StringLookup(mask_token=None)
user_ids_vocabulary.adapt(data["user_id"].map(str))
user_ids_vocabulary

Metal device set to: Apple M1 Pro


2023-04-28 23:01:53.242697: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


In [14]:
recipe_ids_vocabulary = tf.keras.layers.StringLookup(mask_token=None)
recipe_ids_vocabulary.adapt(data["recipe_id"].map(str))

In [22]:
from typing import Dict, Text

In [42]:
class MovieLensModel(tfrs.Model):
    # We derive from a custom base class to help reduce boilerplate. Under the hood,
    # these are still plain Keras Models.

    def __init__(
        self,
        user_model: tf.keras.Model,
        recipe_model: tf.keras.Model,
        task: tfrs.tasks.Retrieval):
        super().__init__()

        # Set up user and movie representations.
        self.user_model = user_model
        self.recipe_model = recipe_model

        # Set up a retrieval task.
        self.task = task

    def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
        # Define how the loss is computed.

        user_embeddings = self.user_model(features["user_id"])
        recipe_embeddings = self.recipe_model(features["recipe_id"])
        print(user_embeddings.shape)

        return self.task(user_embeddings, recipe_embeddings)


In [43]:
tf_data = tf.data.Dataset.from_tensor_slices(data.recipe_id.values.reshape(-1, 1))
tf_data

<_TensorSliceDataset element_spec=TensorSpec(shape=(1,), dtype=tf.int64, name=None)>

In [44]:
ratings = tf.data.Dataset.from_tensor_slices(dict(data[['user_id', 'recipe_id']]))
ratings

<_TensorSliceDataset element_spec={'user_id': TensorSpec(shape=(), dtype=tf.int64, name=None), 'recipe_id': TensorSpec(shape=(), dtype=tf.int64, name=None)}>

In [55]:
# Define user and movie models.
user_model = tf.keras.Sequential([
    user_ids_vocabulary,
    tf.keras.layers.Embedding(user_ids_vocabulary.vocabulary_size(), 64)
])
recipe_model = tf.keras.Sequential([
    recipe_ids_vocabulary,
    tf.keras.layers.Embedding(recipe_ids_vocabulary.vocabulary_size(), 64)
])

# Define your objectives.
task = tfrs.tasks.Retrieval(metrics=tfrs.metrics.FactorizedTopK(
    tf_data.batch(8).map(recipe_model)
  )
)


In [46]:
ratings.batch(16)

<_BatchDataset element_spec={'user_id': TensorSpec(shape=(None,), dtype=tf.int64, name=None), 'recipe_id': TensorSpec(shape=(None,), dtype=tf.int64, name=None)}>

In [58]:
a = tf.random.uniform((128, 64))
b = tf.random.uniform((128, 64))

In [61]:
task = tfrs.tasks.Retrieval(metrics=tfrs.metrics.FactorizedTopK(
    tf_data.batch(8).map(recipe_model)
  )
)

# Create a dummy dataset
tf_data = tf.data.Dataset.from_tensor_slices({
    "recipe_id": tf.constant([1, 2, 3, 4, 5, 6, 7, 8]),
    "recipe_features": tf.random.uniform((8, 64))
})

# Define the recipe model
recipe_input = tf.keras.Input(shape=(1,), dtype=tf.int32)
recipe_embedding = tf.keras.layers.Embedding(input_dim=10, output_dim=64)(recipe_input)
recipe_model = tf.keras.Model(inputs=recipe_input, outputs=recipe_embedding)

# Call the task on a batch of inputs
a = tf.random.uniform((8,))
b = recipe_model(tf.constant([1, 2, 3, 4, 5, 6, 7, 8]))
task(a, b)

InvalidArgumentError: Exception encountered when calling layer 'retrieval_8' (type Retrieval).

{{function_node __wrapped__BatchMatMulV2_device_/job:localhost/replica:0/task:0/device:GPU:0}} In[0] ndims must be >= 2: 1 [Op:BatchMatMulV2]

Call arguments received by layer 'retrieval_8' (type Retrieval):
  • query_embeddings=tf.Tensor(shape=(8,), dtype=float32)
  • candidate_embeddings=tf.Tensor(shape=(8, 1, 64), dtype=float32)
  • sample_weight=None
  • candidate_sampling_probability=None
  • candidate_ids=None
  • compute_metrics=True
  • compute_batch_metrics=True

In [59]:
task(a, b)

ValueError: Exception encountered when calling layer 'streaming_7' (type Streaming).

in user code:

    File "/Users/vinitkanani/git/personal/kaggle/kaggle/food-recommendation-venv/lib/python3.9/site-packages/tensorflow_recommenders/layers/factorized_top_k.py", line 429, in top_scores  *
        scores, indices = tf.math.top_k(scores, k=k_, sorted=self._sorted)

    ValueError: input must have last dimension >= k = 100 but is 1 for '{{node TopKV2}} = TopKV2[T=DT_FLOAT, sorted=true](MatMul, Minimum)' with input shapes: [?,128,1], [] and with computed input tensors: input[1] = <100>.


Call arguments received by layer 'streaming_7' (type Streaming):
  • queries=tf.Tensor(shape=(128, 64), dtype=float32)
  • k=100

In [48]:
# Create a retrieval model.
model = MovieLensModel(user_model, recipe_model, task)
model.compile(optimizer=tf.keras.optimizers.Adagrad(0.5))

# Train for 3 epochs.
model.fit(ratings.batch(4096), epochs=3)

Epoch 1/3
(None, 64)


ValueError: in user code:

    File "/Users/vinitkanani/git/personal/kaggle/kaggle/food-recommendation-venv/lib/python3.9/site-packages/keras/engine/training.py", line 1284, in train_function  *
        return step_function(self, iterator)
    File "/Users/vinitkanani/git/personal/kaggle/kaggle/food-recommendation-venv/lib/python3.9/site-packages/keras/engine/training.py", line 1268, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/Users/vinitkanani/git/personal/kaggle/kaggle/food-recommendation-venv/lib/python3.9/site-packages/keras/engine/training.py", line 1249, in run_step  **
        outputs = model.train_step(data)
    File "/Users/vinitkanani/git/personal/kaggle/kaggle/food-recommendation-venv/lib/python3.9/site-packages/tensorflow_recommenders/models/base.py", line 68, in train_step
        loss = self.compute_loss(inputs, training=True)
    File "/var/folders/3z/s0qdvzzd2kgc2kdxkfvxy9ym0000gn/T/ipykernel_4601/3312599068.py

In [26]:
# Use brute-force search to set up retrieval using the trained representations.
index = tfrs.layers.factorized_top_k.BruteForce(model.user_model)
index.index_from_dataset(
    movies.batch(100).map(lambda title: (title, model.movie_model(title))))

# Get some recommendations.
_, titles = index(np.array(["42"]))
print(f"Top 3 recommendations for user 42: {titles[0, :3]}")

NameError: name 'MovielensModel' is not defined

In [ ]:
# Train.
model.fit(ratings.batch(4096), epochs=3)

In [ ]:
# Set up retrieval using trained representations.
index = tfrs.layers.ann.BruteForce(model.user_model)
index.index_from_dataset(
    movies.batch(100).map(lambda title: (title, model.movie_model(title)))
)

# Get recommendations.
_, titles = index(np.array(["42"]))
print(f"Recommendations for user 42: {titles[0, :3]}")

In [62]:
import pandas as pd
import tensorflow as tf
import tensorflow_recommenders as tfrs

# Load the dataset
ratings_df = pd.DataFrame({
    "user_id": [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
    "recipe_id": [26059, 58970, 27457, 7492, 26820, 37902, 23321, 979, 43133, 29019],
    "rating": [3, 5, 4, 5, 5, 3, 5, 5, 5, 4]
})

# Create user and recipe feature layers
user_ids = ratings_df["user_id"]
recipe_ids = ratings_df["recipe_id"]
rating = ratings_df["rating"]
user_embedding = tf.keras.Sequential([
    tf.keras.layers.experimental.preprocessing.StringLookup(
        vocabulary=user_ids.astype(str), mask_token=None),
    tf.keras.layers.Embedding(len(user_ids) + 1, 32)
])
recipe_embedding = tf.keras.Sequential([
    tf.keras.layers.experimental.preprocessing.StringLookup(
        vocabulary=recipe_ids.astype(str), mask_token=None),
    tf.keras.layers.Embedding(len(recipe_ids) + 1, 32)
])

# Define the model
class RecommenderModel(tfrs.Model):
  def __init__(self):
    super().__init__()
    self.user_embedding = user_embedding
    self.recipe_embedding = recipe_embedding
    self.rating_model = tf.keras.Sequential([
        tf.keras.layers.Dense(32, activation="relu"),
        tf.keras.layers.Dense(16, activation="relu"),
        tf.keras.layers.Dense(1),
    ])
    self.task = tfrs.tasks.Ranking(
        loss=tf.keras.losses.MeanSquaredError(),
        metrics=[tf.keras.metrics.RootMeanSquaredError()]
    )

  def call(self, features):
    user_embedding = self.user_embedding(features["user_id"])
    recipe_embedding = self.recipe_embedding(features["recipe_id"])
    return self.rating_model(tf.concat([user_embedding, recipe_embedding], axis=1))

  def compute_loss(self, features, training=False):
    ratings = features.pop("rating")
    rating_predictions = self(features)
    return self.task(
        labels=ratings,
        predictions=rating_predictions,
    )

# Instantiate the model and compile it
model = RecommenderModel()
model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.1))

# Prepare the input data
train_dataset = tf.data.Dataset.from_tensor_slices(({
    "user_id": user_ids,
    "recipe_id": recipe_ids
}, rating))

# Train the model
model.fit(train_dataset.shuffle(len(ratings_df)).batch(2), epochs=10)


Epoch 1/10


AttributeError: in user code:

    File "/Users/vinitkanani/git/personal/kaggle/kaggle/food-recommendation-venv/lib/python3.9/site-packages/keras/engine/training.py", line 1284, in train_function  *
        return step_function(self, iterator)
    File "/Users/vinitkanani/git/personal/kaggle/kaggle/food-recommendation-venv/lib/python3.9/site-packages/keras/engine/training.py", line 1268, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/Users/vinitkanani/git/personal/kaggle/kaggle/food-recommendation-venv/lib/python3.9/site-packages/keras/engine/training.py", line 1249, in run_step  **
        outputs = model.train_step(data)
    File "/Users/vinitkanani/git/personal/kaggle/kaggle/food-recommendation-venv/lib/python3.9/site-packages/tensorflow_recommenders/models/base.py", line 68, in train_step
        loss = self.compute_loss(inputs, training=True)
    File "/var/folders/3z/s0qdvzzd2kgc2kdxkfvxy9ym0000gn/T/ipykernel_4601/131772394

In [63]:
import tensorflow as tf
import pandas as pd
import tensorflow_recommenders as tfrs


# Convert data to tf.data.Dataset
def create_dataset(dataframe):
    ratings = dataframe.pop("rating")
    feature_ds = tf.data.Dataset.from_tensor_slices(dict(dataframe))
    label_ds = tf.data.Dataset.from_tensor_slices(ratings)
    return tf.data.Dataset.zip((feature_ds, label_ds))

# Split the dataset into training and testing sets
train_size = int(len(data) * 0.8)
train_dataset = create_dataset(data[:train_size])
test_dataset = create_dataset(data[train_size:])

# Define the retrieval task
task = tfrs.tasks.Retrieval(
    metrics=tfrs.metrics.FactorizedTopK(
        candidates=test_dataset.batch(100).map(recipe_model)
    )
)

# Define the model
user_model = tf.keras.Sequential([
    tf.keras.layers.experimental.preprocessing.StringLookup(
        vocabulary=user_ids, mask_token=None
    ),
    tf.keras.layers.Embedding(len(user_ids) + 1, embedding_dimension)
])
recipe_model = tf.keras.Sequential([
    tf.keras.layers.experimental.preprocessing.StringLookup(
        vocabulary=recipe_ids, mask_token=None
    ),
    tf.keras.layers.Embedding(len(recipe_ids) + 1, embedding_dimension)
])
metrics = tfrs.metrics.FactorizedTopK(
    candidates=test_dataset.batch(100).map(recipe_model)
)
model = RecommenderModel(user_model, recipe_model)
model.compile(optimizer=tf.keras.optimizers.Adagrad(0.1))

# Train the model
model.fit(train_dataset.batch(4096), epochs=10)

# Evaluate the model
model.evaluate(test_dataset.batch(4096), return_dict=True)


ValueError: Missing data for input "input_1". You passed a data dictionary with keys ['index', 'user_id', 'recipe_id']. Expected the following keys: ['input_1']